In [1]:
import nltk
from nltk.corpus import movie_reviews
from nltk.tokenize import word_tokenize
import random


nltk.data.path.append('D:/vscode/code/STAT7008B/Group_work')
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\19083\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# 加载数据

In [2]:
import pandas as pd
data = pd.read_csv('Database/aitools.csv')
print(data["tool"])
training_data_ai = []
for i in range(len(data)):
    training_data_ai.append(("what is "+data["tool"][i]   ,    data["tool"][i] + " is a " + str(data["tool_description"][i])))

0                         Codeium
1                         LeiaPix
2       GPT-3 Playground (OpenAI)
3                      BlueWillow
4                    Character.AI
                  ...            
1007                    Shortwave
1008                       Adstra
1009                Mental Models
1010         AI Listing Architect
1011                      Replika
Name: tool, Length: 1012, dtype: object


# 贝叶斯分类器

In [3]:
# 假设我们有一些预处理后的数据集（问题和分类标签）
training_data = [
    ('AI?', 'ai'),
    ('VR?', 'vr'),
    ('FinTech work?', 'fintech'),
    # 更多的问答数据
]



# 特征提取函数
def extract_features(text):
    words = word_tokenize(text)
    return {word.lower(): True for word in words}

# 准备训练集
training_set = [(extract_features(q), label) for (q, label) in training_data]
# print(training_set)
classifier = nltk.NaiveBayesClassifier.train(training_set)

# 测试聊天机器人
def chatbot_response(user_input):
    features = extract_features(user_input)
    return classifier.classify(features)

# 例子输入
question = "AI tools"
print(chatbot_response(question))
question_type = chatbot_response(question)


ai


# 基于 TF-IDF 的答案检索

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 问题-答案对
# knowledge_base = [
#     ("What is AI?", "Artificial Intelligence (AI) refers to the simulation of human intelligence in machines."),
#     ("What is VR?", "Virtual Reality (VR) is a simulated experience that can be similar to or different from the real world."),
#     ("What is blockchain?", "Blockchain is a decentralized digital ledger that records transactions across many computers.")
# ]
if question_type=="ai":
    knowledge_base = training_data_ai

# 构建 TF-IDF 向量
corpus = [pair[0] for pair in knowledge_base]  # 只用问题作为语料
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(corpus)

def retrieve_answer(question, knowledge_base, tfidf_matrix, vectorizer):
    # 将用户问题转换为 TF-IDF 向量
    question_tfidf = vectorizer.transform([question])
    
    # 计算用户问题与知识库中问题的相似度
    cosine_similarities = cosine_similarity(question_tfidf, tfidf_matrix).flatten()
    
    # 找到相似度最高的问题
    best_match_index = cosine_similarities.argmax()
    
    # 返回相似度最高的问题的答案
    return knowledge_base[best_match_index][1]

# 测试
user_question = "What is Jenni AI?"
answer = retrieve_answer(user_question, knowledge_base, tfidf_matrix, vectorizer)
print(answer)


Jenni AI is a AI writing assistant
